In [ ]:
%load_ext autoreload

%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from sktime.forecasting.base import ForecastingHorizon
from sktime.forecasting.model_evaluation import evaluate
from sktime.forecasting.naive import NaiveForecaster
from sktime.performance_metrics.forecasting.probabilistic import PinballLoss
from sktime.split import (
    ExpandingWindowSplitter,
    temporal_train_test_split,
)
from sktime.utils import plotting
from sktime.utils.plotting import plot_windows

from probafcst.model import BenchmarkForecaster
from probafcst.plotting import plot_quantiles
from probafcst.utils.time import get_forecast_dates

sns.set_theme(style="ticks")

In [ ]:
bikes = pd.read_parquet("../data/bikes.parquet")
load = pd.read_parquet("../data/energy.parquet")

In [ ]:
load.tail()

In [ ]:
# get 3 years as hours


In [ ]:
splitter = ExpandingWindowSplitter(
    fh=ForecastingHorizon(np.arange(1, 24 * 7)),
    initial_window=24 * 365 * 3,  # 3 years
    step_length=24 * 30 * 9,  # 9 months
)
len(list(splitter.split(load)))

In [ ]:
plot_windows(splitter, load)

In [ ]:
y_train, y_test = temporal_train_test_split(load, test_size=24 * 7)
fh = ForecastingHorizon(y_test.index, is_relative=False)
# forecaster = NaiveForecaster(strategy="mean", sp=24, window_length=24 * 7 * 3)
forecaster = BenchmarkForecaster(n_weeks=100)
forecaster.fit(y_train)

quantile_levels = [0.025, 0.25, 0.5, 0.75, 0.975]
y_pred = forecaster.predict_quantiles(fh, alpha=quantile_levels)

In [ ]:
y_pred

In [ ]:
# forecaster = NaiveForecaster(strategy="mean", sp=24 * 7)
results = evaluate(
    forecaster,
    splitter,
    load,
    strategy="refit",
    scoring=PinballLoss(alpha=quantile_levels),
    return_data=True,
)
results

In [ ]:
# plot each forecast period
nrows = min(3, len(results))
for i, row in results.iloc[-nrows:].iterrows():
    plot_quantiles(row.y_test, row.y_pred_quantiles)

In [ ]:
columns = [y_pred[i] for i in y_pred.columns]
fig, ax = plotting.plot_series(y_test, *columns)

In [ ]:
loss = PinballLoss(score_average=True)
loss(y_test, y_pred)

In [ ]:
# make a submission

timestamps, forecast_hours = get_forecast_dates(load.index[-1])
print(forecast_hours)
forecaster.fit(load)
fh = ForecastingHorizon(forecast_hours, is_relative=True)
y_pred = forecaster.predict_quantiles(fh, alpha=quantile_levels)
y_pred

In [ ]:
# plot the forecast
columns = [y_pred[i] for i in y_pred.columns]
fig, ax = plotting.plot_series(y_test, *columns)

In [ ]:
bikes = bikes.asfreq("D").interpolate()

In [ ]:
# forecast bikes
fh = np.arange(2, 8)
forecaster = NaiveForecaster(strategy="mean", sp=7, window_length=90)
forecaster.fit(bikes)
y_pred_bikes = forecaster.predict_quantiles(fh, alpha=quantile_levels)

# plot the forecast
columns = [y_pred_bikes[i] for i in y_pred_bikes.columns]
fig, ax = plotting.plot_series(bikes.iloc[-90:], *columns)

In [ ]:
y_pred_bikes

In [ ]:
from probafcst.utils import create_submission

create_submission(energy_preds=y_pred.values, bikes_preds=y_pred_bikes.values)